In [1]:
# MARK APLAWDW
# 取一些 subset 手動標注

In [2]:
import sys
sys.path.append("../")

In [3]:
import torch
import torchaudio
from pathlib import Path
from tgt import write_to_file, TextGrid, IntervalTier, PointTier, Interval, Point
from src.bpdp import bpdp, bp


In [4]:
for wav_fp in sorted(Path("../data/APLAWDW/").glob("**/*.wav")):
    if wav_fp.name.startswith("ac"):
        continue
    # 只取 5 這一組
    if not wav_fp.stem.endswith("5"):
        continue
    
    wav, sr = torchaudio.load(str(wav_fp))
    end_time = wav.shape[-1] / sr

    egg, sr = torchaudio.load(str(wav_fp.with_suffix(".egg")))
    
    # 
    p = bpdp(egg[0].view(-1), sr, wl_0=0.05, wl_1=0.002, f_lo=50.0, f_hi=350.0, beam_size=5, filt="bp1")

    
    # create textgrid
    tg_fp = Path("../data/MAPLAWDW/")/ f"{wav_fp.stem}.TextGrid"
    out_fp = Path("../data/MAPLAWDW/")/ f"{wav_fp.stem}.wav"

    # concat wav, egg
    wav = torch.cat((wav, egg), dim=0)
    torchaudio.save(str(out_fp), wav, sr)

    tg = TextGrid()
    vuv_tier = IntervalTier(0.0, end_time, name="vuv")
    epoch_tier = PointTier(0.0, end_time, name="epoch")

    for p_n in p:
        point = Point(p_n/sr)
        epoch_tier.add_point(point)
    tg.add_tier(vuv_tier)
    tg.add_tier(epoch_tier)

    write_to_file(tg, str(tg_fp))